## <font color='red'> INSTRUCTIONS </font>

<b> 
1. Write your code only in cells below the "WRITE CODE BELOW" title. Do not modify the code below the "DO NOT MODIFY" title. <br>
2. The expected data types of the output answers for each question are given in the last cell through assertion statements. Your answers must match these expected output data types. Hint: Many of the answers need to be a Python dictionary. Consider methods like to_dict() to convert a Pandas Series to a dictionary. <br>
3. The answers are then written to a JSON file named my_results_PA1.json. You can compare this with the provided expected output file "expected_results_PA1.json". <br>
4. After you complete writing your code, click "Kernel -> Restart Kernel and Run All Cells" on the top toolbar. There should NOT be any syntax/runtime errors, otherwise points will be deducted. <br>
5. For submitting your solution, first download your notebook by clicking "File -> Download". Rename the file as &ltTEAM_ID&gt.ipynb" and upload to Canvas.</b>


## <font color='red'> DO NOT MODIFY </font>

In [1]:
import time
import json
import dask
import dask.dataframe as dd
import pandas as pd
import ast
import re
from dask.distributed import Client
import ctypes
import numpy as np

def trim_memory() -> int:
    """
    helps to fix any memory leaks.
    """
    libc = ctypes.CDLL("libc.so.6")
    return libc.malloc_trim(0)

client = Client("127.0.0.1:8786")
client.run(trim_memory)
client = client.restart()
print(client)

None


In [2]:
start = time.time()

## <font color='blue'> WRITE CODE BELOW </font>

In [3]:
# Read user_reviews.csv
user_reviews = dd.read_csv(
    'user_reviews.csv',
    blocksize='64MB',
    usecols=['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText', 'overall', 'summary', 'unixReviewTime', 'reviewTime'],
    dtype={'reviewerID': 'object', 'asin': 'object'}
)

# Read products.csv
products = dd.read_csv(
    'products.csv',
    blocksize='64MB',
    usecols=['asin', 'salesRank', 'imUrl', 'categories', 'title', 'description', 'price', 'related', 'brand'],
    dtype={'asin': 'object'}
)

# Persist DataFrames to avoid reloading costs
user_reviews = user_reviews.persist()
products = products.persist()

# Q1: Percentage of missing values in user_reviews
ans1 = (user_reviews.isnull().sum() / user_reviews.shape[0]).compute() * 100
ans1 = ans1.round(2).to_dict()


# Q2: Percentage of missing values in products
ans2 = (products.isnull().sum() / products.shape[0]).compute() * 100
ans2 = ans2.round(2).to_dict()


# Q3: Pearson correlation between price and overall rating
products['price'] = dd.to_numeric(products['price'], errors='coerce')
user_reviews['overall'] = dd.to_numeric(user_reviews['overall'], errors='coerce')
products = products.persist()
user_reviews = user_reviews.persist()

merged = user_reviews.merge(products[['asin', 'price']], on='asin', how='inner') \
                     .dropna(subset=['price', 'overall']) \
                     .persist()

correlation_matrix = merged[['price', 'overall']].corr().compute()
ans3 = float(correlation_matrix.loc['price', 'overall'])


# Q4: Price statistics in products
price_clean = products['price'].dropna().persist()

# Compute summary stats
price_stats = price_clean.describe().compute()

# MEDIAN CALCUATION NEEDS IMPROVEMENT, CURRENTLY SOMEWHAT INACCURATE (could be the dropna above?)
price_clean_pd = price_clean.compute()
median_price = price_clean_pd.median()

# Convert to plain float
ans4 = {
    'mean': float(price_stats['mean']),
    'std': float(price_stats['std']),
    'min': float(price_stats['min']),
    'max': float(price_stats['max']),
    'median': float(median_price)
}

# MOSTLY CORRECT, BUT HAS AN EXTRA COLUMN THAT IS EMPTY key '' with value 194914 under Beauty 
# Q5: Number of products per super-category (sorted descending)
def extract_super_category(cat_string):
    try:
        categories_list = ast.literal_eval(cat_string)
        if isinstance(categories_list, list) and len(categories_list) > 0:
            return categories_list[0][0]
    except Exception:
        return None

products['super_category'] = products['categories'].map(extract_super_category, meta=('super_category', str))
super_category_counts = products['super_category'].dropna().value_counts().compute()
# Sort by counts descending
super_category_counts = super_category_counts.sort_values(ascending=False)
ans5 = super_category_counts.to_dict()

# Q6: TODO currently has placeholder value
ans6 = 1 

# Q7: TODO currently has placeholder value 
ans7 = 1 

## <font color='red'> DO NOT MODIFY </font>

In [4]:
end = time.time()

In [5]:
print(f"execution time = {end-start}s")

## ADDED CODE, REMOVE LATER
print("ANS1:", ans1)
print("ANS2:", ans2)
print("ANS3:", ans3)
print("ANS4:", ans4)
print("ANS5:", ans5)
print("ANS6:", ans6)
print("ANS7:", ans7)

execution time = 287.0076699256897s
ANS1: {'reviewerID': 0.0, 'asin': 0.0, 'reviewerName': 3.29, 'helpful': 0.0, 'reviewText': 0.02, 'overall': 0.0, 'summary': 0.01, 'unixReviewTime': 0.0, 'reviewTime': 0.0}
ANS2: {'asin': 0.0, 'salesRank': 26.02, 'imUrl': 1.8, 'categories': 0.8, 'title': 15.19, 'description': 41.72, 'price': 35.7, 'related': 29.04, 'brand': 81.69}
ANS3: -0.012821317097412719
ANS4: {'mean': 34.937356116908504, 'std': 71.2636924987751, 'median': 14.99, 'min': 0.0, 'max': 999.99}
ANS5: {'Books': 2369910, 'Clothing, Shoes & Jewelry': 1435868, 'Sports & Outdoors': 529989, 'Electronics': 495476, 'CDs & Vinyl': 491713, 'Home & Kitchen': 436957, 'Cell Phones & Accessories': 346720, 'Toys & Games': 335487, 'Automotive': 331049, 'Digital Music': 270370, 'Tools & Home Improvement': 268469, 'Health & Personal Care': 262317, 'Beauty': 259170, '': 194914, 'Movies & TV': 194874, 'Grocery & Gourmet Food': 171751, 'Office Products': 134281, 'Arts, Crafts & Sewing': 117403, 'Pet Suppli

In [6]:
# DO NOT MODIFY
assert type(ans1) == dict, f"answer to question 1 must be a dictionary like {{'reviewerID':0.2, ..}}, got type = {type(ans1)}"
assert type(ans2) == dict, f"answer to question 2 must be a dictionary like {{'asin':0.2, ..}}, got type = {type(ans2)}"
assert type(ans3) == float, f"answer to question 3 must be a float like 0.8, got type = {type(ans3)}"
assert type(ans4) == dict, f"answer to question 4 must be a dictionary like {{'mean':0.4,'max':0.6,'median':0.6...}}, got type = {type(ans4)}"
assert type(ans5) == dict, f"answer to question 5 must be a dictionary, got type = {type(ans5)}"         
assert ans6 == 0 or ans6==1, f"answer to question 6 must be 0 or 1, got value = {ans6}" 
assert ans7 == 0 or ans7==1, f"answer to question 7 must be 0 or 1, got value = {ans7}" 

ans_dict = {
    "q1": ans1,
    "q2": ans2,
    "q3": ans3,
    "q4": ans4,
    "q5": ans5,
    "q6": ans6,
    "q7": ans7,
    "runtime": end-start
}
with open('my_results_PA1.json', 'w') as outfile: json.dump(ans_dict, outfile)         